# Workplace for: Customer: 1; DFU: Рис длиннозерный 486 гр

In [50]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from lightgbm import LGBMRegressor

In [51]:
df = pd.read_csv("data_grouped/train_group_1_Рис длиннозерный 486 гр.csv")
sales_2_df_t = pd.read_csv("data_test_grouped/test_group_1_Рис длиннозерный 486 гр.csv")
df, sales_2_df_t

(                         DFU  Customer      Period  BPV  Total Sell-in  \
 0    Рис длиннозерный 486 гр         1  2022-01-24  NaN            NaN   
 1    Рис длиннозерный 486 гр         1  2022-01-31  NaN            NaN   
 2    Рис длиннозерный 486 гр         1  2022-02-07  NaN            NaN   
 3    Рис длиннозерный 486 гр         1  2022-02-14  NaN            NaN   
 4    Рис длиннозерный 486 гр         1  2022-02-21  NaN            NaN   
 ..                       ...       ...         ...  ...            ...   
 240  Рис длиннозерный 486 гр         1  2021-12-20  NaN            NaN   
 241  Рис длиннозерный 486 гр         1  2021-12-27  NaN            NaN   
 242  Рис длиннозерный 486 гр         1  2022-01-03  NaN            NaN   
 243  Рис длиннозерный 486 гр         1  2022-01-10  NaN            NaN   
 244  Рис длиннозерный 486 гр         1  2022-01-17  NaN            NaN   
 
      Season  Type Geography End of Period  BPV_sale_period  SoD_sale_period  
 0    Winter  Сеть 

## Vusualization

In [52]:
unique_stat_colors = {
    'BPV Mean': '#1f77b4',
    'BPV Median': '#ff7f0e',
    'BPV CI Low (95%)': '#2ca02c',
    'BPV CI Up (95%)': '#d62728',
    'Total Sell-in Mean': '#9467bd',
    'Total Sell-in Median': '#8c564b',
    'Total Sell-in CI Low (95%)': '#e377c2',
    'Total Sell-in CI Up (95%)': '#7f7f7f',
}

for (dfu, customer), group in df.groupby(['DFU', 'Customer']):
    melted_group = group.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                              var_name='Metric', value_name='Value')

    stat_lines = []

    for metric in ['BPV', 'Total Sell-in']:
        data = group[metric].dropna()
        if not data.empty:
            mean_val = data.mean()
            median_val = data.median()
            std_val = data.std()

            stat_lines.extend([
                (metric, f'{metric} Mean: {mean_val:.2f}', mean_val, unique_stat_colors[f'{metric} Mean']),
                (metric, f'{metric} Median: {median_val:.2f}', median_val, unique_stat_colors[f'{metric} Median'])
            ])

    fig = px.line(
        melted_group,
        x='Period', y='Value', color='Metric',
        title=f'DFU: {dfu} | Customer: {customer}',
        labels={'Value': 'Sales', 'Period': 'Date'}
    )

    for metric, label, value, color in stat_lines:
        fig.add_trace(go.Scatter(
            x=[group['Period'].min(), group['Period'].max()],
            y=[value, value],
            mode='lines',
            name=label,
            line=dict(dash='dot', color=color, width=1.5),
            showlegend=True
        ))

    for metric in ['BPV', 'Total Sell-in']:
        metric_df = group[['Period', metric]].dropna()
        if not metric_df.empty:
            metric_df = metric_df.copy()
            metric_df['Period_ordinal'] = pd.to_datetime(metric_df['Period']).map(pd.Timestamp.toordinal)
            X = metric_df[['Period_ordinal']]
            y = metric_df[metric]
            model = LinearRegression().fit(X, y)
            y_pred = model.predict(X)

            fig.add_trace(go.Scatter(
                x=metric_df['Period'],
                y=y_pred,
                mode='lines',
                name=f'Trend - {metric}',
                line=dict(dash='dash', color='orange')
            ))

    if group[['BPV', 'Total Sell-in']].dropna().shape[0] > 1:
        corr = group['BPV'].corr(group['Total Sell-in'])
        fig.add_annotation(
            text=f"Корреляция BPV и Total Sell-in: {corr:.2f}",
            xref="paper", yref="paper",
            x=0.99, y=1.05, showarrow=False,
            font=dict(size=12, color="white"),
            align="right",
            bordercolor="white", borderwidth=1
        )

    filtered_sales_2 = sales_2_df_t[(sales_2_df_t['DFU'] == dfu) & (sales_2_df_t['Customer'] == customer)]
    if not filtered_sales_2.empty:
        melted_sales_2 = filtered_sales_2.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                                               var_name='Metric', value_name='Value')
        for metric in melted_sales_2['Metric'].unique():
            metric_data = melted_sales_2[melted_sales_2['Metric'] == metric]
            color = 'black' if metric == 'BPV' else 'gray'
            fig.add_trace(go.Scatter(
                x=metric_data['Period'],
                y=metric_data['Value'],
                mode='lines+markers',
                name=f'test_sales - {metric}',
                line=dict(color=color, width=2, dash='dot')
            ))

    fig.update_xaxes(title_text='Дата', tickformat='%Y-%m-%d')
    fig.update_yaxes(title_text='Значение')
    fig.show()


## Machine learning

In [53]:
def compute_metrics(y_true, y_pred, m=1):
    """
    Compute forecast accuracy metrics: WAPE, MAE, RMSE, MASE.

    Parameters
    ----------
    y_true : array-like of shape (n,)
        Истинные значения.
    y_pred : array-like of shape (n,)
        Предсказанные значения.
    m : int, default=1
        Шаг «наивного» прогноза для расчёта MASE.
        Для несезонного ряда обычно m=1;
        для сезонного — период сезонности.

    Returns
    -------
    dict
        {
            'WAPE': float,  # в процентах
            'MAE': float,
            'RMSE': float,
            'MASE': float
        }
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    # WAPE
    num = np.abs(y_true - y_pred).sum()
    denom = np.abs(y_true).sum()
    wape = num / denom * 100 if denom != 0 else np.nan

    # MAE
    mae = np.mean(np.abs(y_true - y_pred))

    # RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    # MASE
    if n > m:
        naive_errors = np.abs(y_true[m:] - y_true[:-m])
        scale = np.mean(naive_errors)
        mase = mae / scale if scale != 0 else np.nan
    else:
        mase = np.nan

    return {
        'WAPE': wape,
        'MAE': mae,
        'RMSE': rmse,
        'MASE': mase
    }

### Загрузка и предобработка данных

- Загружаем обучающую и тестовую выборки по заданным путям.
- Ограничиваем обучающие данные временным интервалом с **13 сентября 2019** по **17 января 2022** — это может быть связано с актуальностью или полнотой данных за этот период.
- В тестовой выборке переименовываем столбец `BPV` в `BPV_true` для дальнейшего сравнения с предсказаниями модели.

In [54]:
train_path = "data_grouped/train_group_1_Рис длиннозерный 486 гр.csv"
test_path = "data_test_grouped/test_group_1_Рис длиннозерный 486 гр.csv"

train_full = pd.read_csv(train_path)
train_full = train_full[
    (train_full["Period"] >= "2019-09-13") &
    (train_full["Period"] <= "2022-01-17")
    ].copy()

test = pd.read_csv(test_path)
test = test.rename(columns={"BPV": "BPV_true"})

### Обработка дат и создание временных признаков

- Преобразуем столбец `Period` в формат `datetime`, чтобы можно было проводить операции с датами.
- Добавляем числовой признак `Period_ord` — порядковый номер даты (количество дней с начала эпохи), который можно использовать в моделях.
- Создаём признак `Period_last_year`, указывающий на аналогичную неделю год назад (вычитанием 52 недель). Этот признак будет полезен для моделирования сезонности и сравнения с данными за прошлый год.

In [55]:
train_full["Period"] = pd.to_datetime(train_full["Period"])
test["Period"] = pd.to_datetime(test["Period"])

train_full["Period_ord"] = train_full["Period"].map(pd.Timestamp.toordinal)
test["Period_ord"] = test["Period"].map(pd.Timestamp.toordinal)

train_full["Period_last_year"] = train_full["Period"] - pd.DateOffset(weeks=52)
test["Period_last_year"] = test["Period"] - pd.DateOffset(weeks=52)

### Добавление продаж за аналогичный период прошлого года

- Создаём вспомогальную таблицу `sellin_map`, в которой переименовываем:
  - `Period` → `Period_last_year`
  - `Total Sell-in` → `sellin_last_year`
- Объединяем `train_full` и `test` с этой таблицей по колонке `Period_last_year`, чтобы добавить информацию о продажах (`Total Sell-in`) за аналогичный период год назад.
- Пропущенные значения (в случае отсутствия данных за прошлый год) заполняем нулями.

In [56]:
sellin_map = train_full[["Period", "Total Sell-in"]].copy().rename(columns={
    "Period": "Period_last_year",
    "Total Sell-in": "sellin_last_year"
})

train_full = train_full.merge(sellin_map, on="Period_last_year", how="left")
train_full["sellin_last_year"] = train_full["sellin_last_year"].fillna(0)

test = test.merge(sellin_map, on="Period_last_year", how="left")
test["sellin_last_year"] = test["sellin_last_year"].fillna(0)

### Создание новых признаков на основе BPV и SoD

- `bpv_vs_sod_flag` — бинарный признак: равен 1, если продажи BPV в периоде превышают продажи SoD, иначе 0.
- `bpv_minus_sod` — абсолютная разница между продажами BPV и SoD.
- `bpv_div_sod` — отношение продаж BPV к продажам SoD; добавляется небольшая константа к знаменателю, чтобы избежать деления на ноль.
- `month` — номер месяца, извлечённый из даты. Может быть полезен для учёта сезонности.
- Сортировка данных по времени (`Period_ord`) и сброс индекса.
- `bpv_trend_flip` — бинарный признак разворота тренда: равен 1, если направление изменения BPV изменилось по сравнению с предыдущим периодом (например, рост сменился падением).

In [57]:
train_full["bpv_vs_sod_flag"] = (train_full["BPV_sale_period"] > train_full["SoD_sale_period"]).astype(int)
train_full["bpv_minus_sod"] = train_full["BPV_sale_period"] - train_full["SoD_sale_period"]
train_full["bpv_div_sod"] = train_full["BPV_sale_period"] / (train_full["SoD_sale_period"] + 1e-9)
train_full["month"] = train_full["Period"].dt.month

train_full = train_full.sort_values("Period_ord").reset_index(drop=True)
delta_bpv = train_full["BPV"].diff()
bpv_trend_flip = (np.sign(delta_bpv) != np.sign(delta_bpv.shift(1))).astype(int)
train_full["bpv_trend_flip"] = bpv_trend_flip.fillna(0)

### Обнаружение пиков и подготовка данных к моделированию

- `is_after_peak` — бинарный признак, равный 1, если текущий период следует сразу за всплеском продаж (то есть, два периода назад BPV было 0, а в предыдущем — положительное значение).
- Создаётся копия обучающей выборки `train_full_iter` для дальнейших итеративных преобразований или прогнозирования.
- `pred_dates` — сохраняются даты, для которых отсутствуют значения BPV (если такие имеются), чтобы затем сделать по ним предсказания.
- Удаляются нечисловые или неиспользуемые признаки (`Customer`, `DFU`, `Period`, `Period_last_year`), если они присутствуют в таблице.

In [58]:
is_after_peak = [0, 0]
for i in range(2, len(train_full)):
    prev = train_full.loc[i - 1, "BPV"]
    prev_prev = train_full.loc[i - 2, "BPV"]
    is_after_peak.append(int(prev > 0 and prev_prev == 0))
train_full["is_after_peak"] = is_after_peak

train_full_iter = train_full.copy()
pred_dates = train_full_iter.loc[train_full_iter["BPV"].isna(), "Period"]

drop_cols = ["Customer", "DFU", "Period", "Period_last_year"]
train_full_iter = train_full_iter.drop(columns=[c for c in drop_cols if c in train_full_iter.columns])

### Формирование обучающей выборки и инициализация модели

- `feature_cols` — список признаков, которые будут использоваться для обучения модели.
- `train_df` — подмножество обучающей таблицы, где значение целевой переменной `BPV` известно.
- `X_train` — матрица признаков для обучения.
- `y_train` — целевая переменная, логарифмированная с помощью `np.log1p` для стабилизации разброса и улучшения качества регрессии (особенно полезно при наличии сильной положительной асимметрии в распределении).
- Инициализируется модель градиентного бустинга `LGBMRegressor` с заданными параметрами:
  - `learning_rate=0.1` — скорость обучения,
  - `n_estimators=200` — число деревьев,
  - `max_depth=3` — максимальная глубина дерева,
  - `num_leaves=15` — ограничение на количество листьев,
  - `random_state=42` — фиксированное значение для воспроизводимости результатов.

In [59]:
feature_cols = [
    "Period_ord", "BPV_sale_period", "SoD_sale_period", "sellin_last_year",
    "bpv_vs_sod_flag", "bpv_minus_sod", "bpv_div_sod", "month",
    "bpv_trend_flip", "is_after_peak"
]

train_df = train_full_iter[train_full_iter["BPV"].notna()].reset_index(drop=True)
X_train = train_df[feature_cols]
y_train = np.log1p(train_df["BPV"].values)

model = LGBMRegressor(
    learning_rate=0.1,
    n_estimators=200,
    max_depth=3,
    num_leaves=15,
    random_state=42
)

### Обучение модели

Модель `LGBMRegressor` обучается на признаках `X_train` и целевой переменной `y_train`, логарифмированной с помощью `log1p`.
Это позволяет модели учитывать относительные изменения в значении BPV и лучше справляться с выбросами и неравномерным распределением.

In [60]:
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 93, number of used features: 9
[LightGBM] [Info] Start training from score 2.321464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMRegressor(max_depth=3, n_estimators=200, num_leaves=15, random_state=42)

### Итеративное прогнозирование значений BPV

- `pred_idx` — индексы строк, в которых отсутствует значение `BPV` (предположительно — периоды будущего или для прогнозирования).
- Устанавливаются параметры:
  - `threshold` — порог, выше которого применяется корректировка прогноза,
  - `boost_coef` — коэффициент усиления прогноза при превышении порога.

Далее выполняется цикл по всем индексам, требующим предсказания:

1. Извлекаются значения `BPV` за предыдущие два периода.
2. Вычисляются признаки `bpv_trend_flip` и `is_after_peak` на основе истории — они могут изменяться при переходе к следующему шагу, поэтому пересчитываются вручную.
3. Формируется входная строка признаков и передаётся модели для получения лог-прогноза.
4. Прогноз возвращается в исходный масштаб с помощью `expm1`.
5. Применяются корректировки:
   - Если прогноз превышает порог, он масштабируется на коэффициент `boost_coef`.
   - Если период следует за пиком (`is_after_peak = 1`), прогноз уменьшается до уровня предыдущего значения минус 10.
   - Если `BPV_sale_period == 0`, прогноз принудительно обнуляется.
6. Результат записывается обратно в таблицу `train_full_iter`.

In [61]:
pred_idx = train_full_iter[train_full_iter["BPV"].isna()].index.tolist()
threshold = 40
boost_coef = 1.4

for idx in pred_idx:
    row = train_full_iter.loc[idx]

    prev_bpv_1 = train_full_iter.loc[idx - 1, "BPV"] if idx > 0 else 0
    prev_bpv_2 = train_full_iter.loc[idx - 2, "BPV"] if idx > 1 else 0

    bpv_trend_flip = int(np.sign(prev_bpv_1 - prev_bpv_2) != np.sign(row["BPV_sale_period"] - prev_bpv_1))
    is_after_peak = int(prev_bpv_1 > 0 and prev_bpv_2 == 0)

    input_row = row[feature_cols].copy()
    input_row["bpv_trend_flip"] = bpv_trend_flip
    input_row["is_after_peak"] = is_after_peak
    input_df = pd.DataFrame([input_row])

    log_pred = model.predict(input_df)[0]
    pred = np.expm1(log_pred)

    if pred > threshold:
        pred *= boost_coef
    if is_after_peak:
        pred = max(prev_bpv_1 - 10, 0)

    pred = 0 if row["BPV_sale_period"] == 0 else pred
    train_full_iter.at[idx, "BPV"] = max(pred, 0)

### Оценка качества прогноза

- `y_pred_full` — финальные предсказанные значения `BPV` для нужных периодов.
- Формируется таблица `merged`, объединяющая предсказания (`BPV_pred`) с реальными значениями (`BPV_true`) по дате (`Period`).
- Пропущенные значения в `BPV_true` (если их не оказалось в тестовой выборке) заполняются нулями.
- Вызывается функция `compute_metrics`, которая рассчитывает метрики качества модели:
  - **WAPE** (Weighted Absolute Percentage Error),
  - **MAE** (Mean Absolute Error),
  - **RMSE** (Root Mean Squared Error),
  - **MASE** (Mean Absolute Scaled Error), параметр `m=1` соответствует недельной сезонности.

Метрики выводятся в читаемом формате для анализа точности модели.

In [62]:
y_pred_full = train_full_iter.loc[pred_idx, "BPV"].values
merged = pd.DataFrame({
    "Period": pred_dates.reset_index(drop=True),
    "BPV_pred": y_pred_full
})
merged = merged.merge(test[["Period", "BPV_true"]], on="Period", how="left")
merged["BPV_true"] = merged["BPV_true"].fillna(0)

metrics = compute_metrics(
    merged["BPV_true"],
    merged["BPV_pred"],
    m=1
)

print(f"WAPE = {metrics['WAPE']:.2f}%")
print(f"MAE  = {metrics['MAE']:.4f}")
print(f"RMSE = {metrics['RMSE']:.4f}")
print(f"MASE = {metrics['MASE']:.4f}")

WAPE = 24.19%
MAE  = 3.8460
RMSE = 7.9803
MASE = 0.2257


### Сохранение предсказаний

- Удаляется столбец `BPV_true`, чтобы в итоговом файле остались только дата (`Period`) и предсказанное значение (`BPV_pred`).
- Результаты сохраняются в CSV-файл с именем `1_Рис длиннозерный 486_гр_predictions.csv` без индекса.

In [63]:
to_save = merged.drop(columns=["BPV_true"])
to_save.to_csv("1_Рис длиннозерный 486_гр_eval_predictions.csv", index=False)

In [64]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_true"],
    mode="markers+lines",
    name="BPV_true",
    line=dict(color="blue"),
    marker=dict(size=6)
))
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_pred"],
    mode="markers+lines",
    name="BPV_pred",
    line=dict(color="red", dash="dot"),
    marker=dict(size=6)
))
fig.update_layout(
    title="1 Рис длиннозерный 486 гр. Сравнение BPV_true и BPV_pred",
    xaxis_title="Period",
    yaxis_title="BPV",
    legend_title="Легенда"
)
fig.show()

Prediction for 1.5 years using previous pipeline

In [65]:
train_path = "data_grouped/train_group_1_Рис длиннозерный 486 гр.csv"
test_path = "data_test_grouped/test_group_1_Рис длиннозерный 486 гр.csv"

train_full = pd.read_csv(train_path)
train_full = train_full[
    (train_full["Period"] >= "2019-09-13")
    ].copy()

test = pd.read_csv(test_path)
test = test.rename(columns={"BPV": "BPV_true"})

In [66]:
train_full["Period"] = pd.to_datetime(train_full["Period"])
test["Period"] = pd.to_datetime(test["Period"])

train_full["Period_ord"] = train_full["Period"].map(pd.Timestamp.toordinal)
test["Period_ord"] = test["Period"].map(pd.Timestamp.toordinal)

train_full["Period_last_year"] = train_full["Period"] - pd.DateOffset(weeks=52)
test["Period_last_year"] = test["Period"] - pd.DateOffset(weeks=52)

In [67]:
sellin_map = train_full[["Period", "Total Sell-in"]].copy().rename(columns={
    "Period": "Period_last_year",
    "Total Sell-in": "sellin_last_year"
})

train_full = train_full.merge(sellin_map, on="Period_last_year", how="left")
train_full["sellin_last_year"] = train_full["sellin_last_year"].fillna(0)

test = test.merge(sellin_map, on="Period_last_year", how="left")
test["sellin_last_year"] = test["sellin_last_year"].fillna(0)

In [68]:
train_full["bpv_vs_sod_flag"] = (train_full["BPV_sale_period"] > train_full["SoD_sale_period"]).astype(int)
train_full["bpv_minus_sod"] = train_full["BPV_sale_period"] - train_full["SoD_sale_period"]
train_full["bpv_div_sod"] = train_full["BPV_sale_period"] / (train_full["SoD_sale_period"] + 1e-9)
train_full["month"] = train_full["Period"].dt.month

train_full = train_full.sort_values("Period_ord").reset_index(drop=True)
delta_bpv = train_full["BPV"].diff()
bpv_trend_flip = (np.sign(delta_bpv) != np.sign(delta_bpv.shift(1))).astype(int)
train_full["bpv_trend_flip"] = bpv_trend_flip.fillna(0)

In [69]:
is_after_peak = [0, 0]
for i in range(2, len(train_full)):
    prev = train_full.loc[i - 1, "BPV"]
    prev_prev = train_full.loc[i - 2, "BPV"]
    is_after_peak.append(int(prev > 0 and prev_prev == 0))
train_full["is_after_peak"] = is_after_peak

train_full_iter = train_full.copy()
pred_dates = train_full_iter.loc[train_full_iter["BPV"].isna(), "Period"]

drop_cols = ["Customer", "DFU", "Period", "Period_last_year"]
train_full_iter = train_full_iter.drop(columns=[c for c in drop_cols if c in train_full_iter.columns])

In [70]:
feature_cols = [
    "Period_ord", "BPV_sale_period", "SoD_sale_period", "sellin_last_year",
    "bpv_vs_sod_flag", "bpv_minus_sod", "bpv_div_sod", "month",
    "bpv_trend_flip", "is_after_peak"
]

train_df = train_full_iter[train_full_iter["BPV"].notna()].reset_index(drop=True)
X_train = train_df[feature_cols]
y_train = np.log1p(train_df["BPV"].values)

model = LGBMRegressor(
    learning_rate=0.1,
    n_estimators=200,
    max_depth=3,
    num_leaves=15,
    random_state=42
)

In [71]:
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000110 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 93, number of used features: 9
[LightGBM] [Info] Start training from score 2.321464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMRegressor(max_depth=3, n_estimators=200, num_leaves=15, random_state=42)

In [72]:
pred_idx = train_full_iter[train_full_iter["BPV"].isna()].index.tolist()
threshold = 40
boost_coef = 1.4

for idx in pred_idx:
    row = train_full_iter.loc[idx]

    prev_bpv_1 = train_full_iter.loc[idx - 1, "BPV"] if idx > 0 else 0
    prev_bpv_2 = train_full_iter.loc[idx - 2, "BPV"] if idx > 1 else 0

    bpv_trend_flip = int(np.sign(prev_bpv_1 - prev_bpv_2) != np.sign(row["BPV_sale_period"] - prev_bpv_1))
    is_after_peak = int(prev_bpv_1 > 0 and prev_bpv_2 == 0)

    input_row = row[feature_cols].copy()
    input_row["bpv_trend_flip"] = bpv_trend_flip
    input_row["is_after_peak"] = is_after_peak
    input_df = pd.DataFrame([input_row])

    log_pred = model.predict(input_df)[0]
    pred = np.expm1(log_pred)

    if pred > threshold:
        pred *= boost_coef
    if is_after_peak:
        pred = max(prev_bpv_1 - 10, 0)

    pred = 0 if row["BPV_sale_period"] == 0 else pred
    train_full_iter.at[idx, "BPV"] = max(pred, 0)

In [73]:
y_pred_full = train_full_iter.loc[pred_idx, "BPV"].values
result = pd.DataFrame({
    "Period": pred_dates.reset_index(drop=True),
    "BPV_pred": y_pred_full
})

In [74]:
result.to_csv("1_Рис длиннозерный 486_гр_predictions.csv", index=False)